In [ ]:
%pip install ultralytics
%pip install matplotlib
%pip install numpy
import ultralytics
ultralytics.checks()

**CONSTANT VARIABLE**

In [ ]:
yaml_file_path='/kaggle/input/v3-dataset/data.yaml'
model_path = '/kaggle/working/runs/detect/train/'
best_model_path ='/kaggle/working/runs/detect/train/weights/best.pt'
img_size= 800
confidence_val = 0.25
epoch_num= 100
learning_rate = 0.0002

In [ ]:
from ultralytics import YOLO

# Load a model
model = YOLO("yolov8m.pt")  # build a new model from scratch

# Use the model
results = model.train(data=yaml_file_path, imgsz=img_size, epochs=epoch_num, lr0= learning_rate)  # train the model

In [ ]:
def print_attributes(obj, indent=0, ignore_attribute_arr=[]):
    for attribute, value in obj.__dict__.items():
        # skip the ignore value
        if(attribute not in ignore_attribute_arr):
            if hasattr(value, '__dict__'):  # Check if the attribute is an object with attributes
                print(' ' * indent + f"{attribute}:")
                print_attributes(value, indent + 4, ignore_attribute_arr)
            else:
                print(' ' * indent + f"{attribute}: {value}")

In [ ]:
import numpy as np
train_result = results
confusion_matrix = train_result.confusion_matrix.matrix
sum_diagonal = np.trace(confusion_matrix)
sum_all_elements = np.sum(confusion_matrix[:4,])

# Step 5: Compute the accuracy
accuracy = sum_diagonal / sum_all_elements

# print attribute from result 
print_attributes(train_result, 2, ['p_curve', 'r_curve', 'f1_curve', 'px', 'prec_values', 'all_ap'])
# Step 6: Print the result
print("Sum of all correct prediction:", sum_diagonal)
print("Sum of all elements in the matrix:", sum_all_elements)
print("Accuracy of train_val_result:", accuracy)


In [ ]:
import matplotlib.pyplot as plt


def plot(ls, size):
    c=1
    plt.figure(figsize=(90,40))
    for im in ls:
        plt.subplot(size[0],size[1],c)
        im = plt.imread(model_path+im)
        plt.imshow(im)
        c+=1
    plt.show()
plot(['labels.jpg', 'labels_correlogram.jpg', 'results.png' ], (3,1))

In [ ]:
import matplotlib.pyplot as plt

# model_path = '/kaggle/working/runs/detect/train/'

def plot(ls, size):
    c=1
    plt.figure(figsize=(90,40))
    for im in ls:
        plt.subplot(size[0],size[1],c)
        im = plt.imread(model_path+im)
        plt.imshow(im)
        c+=1
    plt.show()

plot(['R_curve.png', 'F1_curve.png','PR_curve.png', 'confusion_matrix.png', 
     'confusion_matrix_normalized.png'], (5,1))

# plot(['results.png'],(1,1))

# **Validate**

In [ ]:
# validate the dataset
from ultralytics import YOLO
model = YOLO(best_model_path)

# Customize validation settings
validation_results = model.val(data=yaml_file_path, imgsz=img_size, split='val', conf=confidence_val)

In [ ]:
import numpy as np
confusion_matrix = validation_results.confusion_matrix.matrix
sum_diagonal = np.trace(confusion_matrix)
sum_all_elements = np.sum(confusion_matrix[:, :5])

# print attribute from result 
print_attributes(validation_results, 2, ['p_curve', 'r_curve', 'f1_curve', 'px', 'prec_values', 'all_ap'])
# Step 5: Compute the accuracy
accuracy = sum_diagonal / sum_all_elements

# Step 6: Print the result
print("Sum of all correct prediction:", sum_diagonal)
print("Sum of all elements in the matrix:", sum_all_elements)
print("Accuracy of validation:", accuracy)

# **TEST**

In [ ]:
from ultralytics import YOLO
model = YOLO(best_model_path)

# Customize validation settings
test_results = model.val(data=yaml_file_path, imgsz=img_size, split='test', conf=confidence_val)
test_results

In [ ]:
import numpy as np
confusion_matrix = test_results.confusion_matrix.matrix
sum_diagonal = np.trace(confusion_matrix)
sum_all_elements = np.sum(confusion_matrix[:, :5])

# Step 5: Compute the accuracy
accuracy = sum_diagonal / sum_all_elements
print_attributes(test_results, 2, ['p_curve', 'r_curve', 'f1_curve', 'px', 'prec_values', 'all_ap'])

# Step 6: Print the result
print("Sum of all correct prediction:", sum_diagonal)
print("Sum of all elements in the matrix:", sum_all_elements)
print("Accuracy of test_result:", accuracy)